In [2]:
import pandas as pd
from scipy.stats import zscore
import numpy as np
import math

In [3]:
df = pd.read_csv('df_master.csv', index_col = 0)

/Users/leotsang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#exclude missing pos value

df = df[df['pos'].notnull()]
len(df)

51410

In [5]:
#add the ht_in into our whole dataset
df['ht'] = df['ht'].astype(str)

ht_in = []

for i in df['ht']:
    height = i.split('-')
    try: #try to put it in inches
        ft = int(height[0])
        inch = int(height[1])
        ht_in.append(ft*12+inch*2)
    except: #null value save as null value
        ht_in.append('NaN')
    
df['ht_in'] = ht_in
df['ht_in'] = df['ht_in'].astype(float)

In [6]:
#add a boolean classifier for drafted players as 1 or non-drafted as 0
drafted = []

for i in df['pick']:
    if i > 0:
        drafted.append(1)
    else:
        drafted.append(0)
        
df['drafted'] = drafted
df['drafted'] = df['drafted'].astype(bool)

In [7]:
#change yr to categorical data
yr_cat = []

for i in df['yr']:
    if i == 'Fr':
        yr_cat.append(1)
    elif i == 'So':
        yr_cat.append(2)
    elif i == 'Jr':
        yr_cat.append(3)
    elif i == 'Sr':
        yr_cat.append(4)
    else:
        yr_cat.append(0)
        
df['yr_cat'] = yr_cat
df['yr_cat'] = df['yr_cat'].astype(int)

In [8]:
df.columns

Index(['player_name', 'school', 'conference', 'GP', 'Min_per', 'ORtg', 'usg',
       'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM',
       'FTA', 'FT_per', '2PM', '2PA', '2P_per', '3PM', '3PA', '3P_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'rec-rk', 'ast/tov', 'rimmade',
       'rimmade + rimmiss', 'midmade', 'midmade + midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+mismiss)', 'dunksmade',
       'dunksmiss + dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'pos', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'ht_in',
       'drafted', 'yr_cat'],
      dtype='object')

In [9]:
df[df['player_name']=='Ben Simmons']

,player_name,school,conference,GP,Min_per,ORtg,usg,eFG,TS_per,ORB_per,...,blk,pts,pos,2PA_pg,2PM_pg,3PA_pg,3PM_pg,ht_in,drafted,yr_cat
44298,Ben Simmons,LSU,SEC,33,86.5,112.9,28.2,56.1,59.95,9.6,...,0.8182,19.1515,Wing F,11.606061,6.515152,0.090909,0.030303,92.0,True,1


In [10]:
 df[df['player_name']=='Zion Williamson']

,player_name,school,conference,GP,Min_per,ORtg,usg,eFG,TS_per,ORB_per,...,blk,pts,pos,2PA_pg,2PM_pg,3PA_pg,3PM_pg,ht_in,drafted,yr_cat
56558,Zion Williamson,Duke,ACC,33,64.9,129.2,28.2,70.8,70.19,12.8,...,1.7879,22.6061,Wing F,11.030303,8.242424,2.151515,0.727273,86.0,True,1


In [11]:
df.groupby('pos')['ht_in'].describe()

,count,mean,std,min,25%,50%,75%,max
pos,,,,,,,,
C,7470.0,89.665596,2.655794,48.0,88.0,90.0,92.0,96.0
Combo G,8137.0,77.220843,3.169812,48.0,74.0,78.0,80.0,92.0
PF/C,9218.0,84.580603,4.982737,48.0,82.0,86.0,88.0,94.0
Pure PG,2436.0,77.258621,3.944641,48.0,74.0,78.0,80.0,94.0
Scoring PG,5360.0,76.999254,3.593031,64.0,74.0,76.0,80.0,94.0
Stretch 4,2826.0,86.328379,3.644332,48.0,84.0,86.0,88.0,94.0
Wing F,4474.0,85.724184,2.622353,72.0,84.0,86.0,88.0,94.0
Wing G,11269.0,80.195581,3.414779,48.0,78.0,80.0,82.0,86.0


In [12]:
def zptile(z_score):
    return .5 * (math.erf(z_score / 2 ** .5) + 1)

In [13]:
#calculating ATH

ht_mean = df.groupby('pos')['ht_in'].transform(np.mean)
ht_std = df.groupby('pos')['ht_in'].transform(np.std)
ht_z = (df['ht_in'] - ht_mean)/ht_std

dreb_mean = df.groupby('pos')['dreb'].transform(np.mean)
dreb_std = df.groupby('pos')['dreb'].transform(np.std)
dreb_z = (df['dreb'] - dreb_mean)/dreb_std

oreb_mean = df.groupby('pos')['oreb'].transform(np.mean)
oreb_std = df.groupby('pos')['oreb'].transform(np.std)
oreb_z = df.groupby('pos')['oreb'].transform(np.std)

ATH = (ht_z.transform(zptile) * 5 + dreb_z.transform(zptile) * 2 + oreb_z.transform(zptile)*3) / 2
ATH = (ATH - np.mean(ATH))/np.std(ATH)
ATH = ATH.transform(zptile) * 5

df['ATH'] = round(ATH * 2) / 2

In [14]:
##ADJ Game Played

GP_adj = []

for i in df['GP']:
    if i < 5:
        GP_adj.append('<5')
    elif i < 10:
        GP_adj.append('<10')
    elif i < 15:
        GP_adj.append('<15')
    elif i < 20: 
        GP_adj.append('<20')
    elif i < 30:
        GP_adj.append('<30')
    else:
        GP_adj.append('30+')
        
df['GP_adj']= GP_adj

In [15]:
#calculate BBIQ

TS_mean = df.groupby('GP_adj')['TS_per'].transform(np.mean)
TS_std = df.groupby('GP_adj')['TS_per'].transform(np.std)
TS_z = (df['TS_per'] - TS_mean)/TS_std

efg_mean = df.groupby('GP_adj')['eFG'].transform(np.mean)
efg_std = df.groupby('GP_adj')['eFG'].transform(np.std)
efg_z = (df['eFG'] - efg_mean)/efg_std

AST_mean = df.groupby('GP_adj')['ast'].transform(np.mean)
AST_std = df.groupby('GP_adj')['ast'].transform(np.std)
AST_z = (df['ast'] - AST_mean)/AST_std

TO_mean = df.groupby('GP_adj')['TO_per'].transform(np.mean)
TO_std = df.groupby('GP_adj')['TO_per'].transform(np.std)
TO_z = -(df['TO_per'] - TO_mean)/TO_std

tpa_mean = df.groupby('GP_adj')['3PA_pg'].transform(np.mean)
tpa_std = df.groupby('GP_adj')['3PA_pg'].transform(np.std)
tpa_z = (df['3PA_pg'] - tpa_mean)/tpa_std

BBIQ = (TS_z.transform(zptile) + efg_z.transform(zptile)+ AST_z.transform(zptile) + TO_z.transform(zptile)+ tpa_z.transform(zptile))
BBIQ = (BBIQ- np.mean(BBIQ) )/ np.std(BBIQ)
BBIQ = BBIQ.transform(zptile) * 5

df['BBIQ'] = round(BBIQ * 2) / 2


In [16]:
df[df['player_name']=='Lonzo Ball']

,player_name,school,conference,GP,Min_per,ORtg,usg,eFG,TS_per,ORB_per,...,2PA_pg,2PM_pg,3PA_pg,3PM_pg,ht_in,drafted,yr_cat,ATH,GP_adj,BBIQ
47421,Lonzo Ball,UCLA,P12,36,87.6,129.4,20.1,66.8,67.26,3.2,...,4.138889,3.027778,5.388889,2.222222,84.0,True,1,5.0,30+,5.0


In [17]:
#Ben Simmons special stats
ht_z[44298], dreb_z[44298], oreb_z[44298], ATH[44298], BBIQ[44298]

(2.3931999867273492,
 3.844290741418651,
 0.7198748487012765,
 4.926370979766814,
 4.227183404195842)

In [19]:
df[(df['drafted']==1) & (df['BBIQ'] == 5)]

,player_name,school,conference,GP,Min_per,ORtg,usg,eFG,TS_per,ORB_per,...,2PA_pg,2PM_pg,3PA_pg,3PM_pg,ht_in,drafted,yr_cat,ATH,GP_adj,BBIQ
116,Charles Jenkins,Hofstra,CAA,32,92.4,123.2,28.7,57.7,62.93,1.6,...,9.906250,5.500000,4.750000,1.968750,78.0,True,4,3.5,30+,5.0
390,E'Twaun Moore,Purdue,B10,34,84.4,112.1,26.8,51.8,54.55,5.1,...,9.382353,4.441176,5.147059,2.058824,80.0,True,4,4.5,30+,5.0
2678,Jordan Crawford,Xavier,A10,35,79.4,113.0,28.9,53.2,56.43,3.4,...,10.314286,5.171429,5.771429,2.257143,80.0,True,2,4.5,30+,5.0
3857,Jon Diebler,Ohio St.,B10,37,88.9,141.9,13.5,70.6,72.33,1.3,...,1.594595,0.837838,6.135135,3.081081,84.0,True,4,4.5,30+,5.0
5590,James Anderson,Oklahoma St.,B12,33,84.8,117.5,30.7,53.2,59.61,6.7,...,8.393939,4.606061,6.575758,2.242424,84.0,True,3,4.5,30+,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52800,Jordan Poole,Michigan,B10,37,82.6,108.4,20.0,53.8,57.32,0.8,...,4.486486,2.324324,5.486486,2.027027,82.0,True,2,4.5,30+,5.0
55351,Cameron Johnson,North Carolina,ACC,36,74.6,128.9,20.7,62.0,64.65,6.5,...,5.777778,3.194444,5.833333,2.666667,90.0,True,4,5.0,30+,5.0
56575,Bol Bol,Oregon,P12,9,17.6,116.5,30.4,61.0,63.18,9.7,...,11.888889,6.777778,2.777778,1.444444,88.0,True,1,4.5,<10,5.0
56676,Darius Garland,Vanderbilt,SEC,5,10.8,110.1,26.9,63.9,65.75,1.6,...,6.200000,3.600000,4.600000,2.200000,76.0,True,1,2.5,<10,5.0


In [323]:
df[df['drafted']==1]['BBIQ'].describe()

count    494.000000
mean       3.987854
std        0.971633
min        0.500000
25%        3.500000
50%        4.500000
75%        4.500000
max        5.000000
Name: BBIQ, dtype: float64

In [324]:
#Zion special stats
ht_z[56558], dreb_z[56558], oreb_z[56558], ATH[56558], BBIQ[56558]

(0.10517874434153482,
 1.7667957958692162,
 0.7198748487012765,
 3.886807929847067,
 4.746282575696432)

In [325]:
#Lonzo Ball Athletic Stats
ATH[47421], BBIQ[47421]

(4.846093290617876, 4.946378999936426)

In [326]:
len(df[df['bpm']>=0])

18192

In [327]:
df['pos'].unique()

array(['Wing G', 'Combo G', 'Pure PG', 'Wing F', 'PF/C', 'C', 'Stretch 4',
       'Scoring PG'], dtype=object)

In [20]:
###Adding Atheleticism
pos = ['Wing G', 'Combo G', 'Pure PG', 'Wing F', 'PF/C','C', 'Stretch 4', 'Scoring PG']

df[df['pos']=='Pure PG'].ht_in.describe()

count    2436.000000
mean       77.258621
std         3.944641
min        48.000000
25%        74.000000
50%        78.000000
75%        80.000000
max        94.000000
Name: ht_in, dtype: float64

In [329]:
#splitting our data to test train split
train = df[df['year']<= 2017]
test = df[(df['year']>=2018) & (df['year']<= 2019)]
df_eval = df[df['year']==2020]

In [330]:
train.to_csv('train.csv', index=True)
test.to_csv('test.csv', index=True)
df_eval.to_csv('pred.csv', index=True)

In [22]:
df['drafted'].unique()

array([False,  True])